# Estimating the Parameters of Differential Equations using Turing.jl

## Load packages

In [1]:
using DifferentialEquations
using ParameterizedFunctions
using RecursiveArrayTools
using Turing
using Plots
srand(31415926) # fix random seed just for demo purpose
;

[Turing]: AD chunk size is set as 40


## Define  a DE problem using DifferentialEquations.jl

Here `a` is the parameter to estimate and `b`, `c` and `d` are fixed, which can be seen from that `a` uses `=>` and others use `=` for value assignment.

> The Lotka–Volterra equations, also known as the predator–prey equations, are a pair of first-order, nonlinear, differential equations frequently used to describe the dynamics of biological systems in which two species interact, one as a predator and the other as prey. - Wikipedia

In [2]:
f = @ode_def_nohes LotkaVolterraTest begin
    dx = a*x - b*x*y
    dy = -c*y + d*x*y
end a=>1.5 b=1.0 c=3.0 d=1.0
;

Set initial states and span

In [3]:
u0 = [1.0;1.0]
tspan = (0.0,10.0)
;

## Generate data using `a=1.5`

In [4]:
prob = ODEProblem(f,u0,tspan)
sol = solve(prob,Tsit5())
t = collect(linspace(0,10,40)) # I reduce the number of data so that I can see error bars from the 2nd prediction way
sig = 0.49  # nosie level
data = convert(Array, VectorOfArray([(sol(t[i]) + sig*randn(2)) for i in 1:length(t)]))
;

In [5]:
plot_data() = begin
    scatter(t, data[1,:], lab="#prey (data)")
    scatter!(t, data[2,:], lab="#predator (data)")
end

plot_data()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #prey (data) 
 
 
 
 
 #predator (data)

Helper function for update parameter

In [6]:
function problem_new_parameters(prob::ODEProblem,p)
  f = (t,u,du) -> prob.f(t,u,p,du)
  uEltype = eltype(p)
  u0 = [uEltype(prob.u0[i]) for i in 1:length(prob.u0)]
  tspan = (uEltype(prob.tspan[1]),uEltype(prob.tspan[2]))
  ODEProblem(f,u0,tspan)
end
;

## Define a Bayes model to fit parameters

$$ a \sim \text{Truncated}(\text{Normal}(1.5, 1), 0.5, 2.5) $$

$$ \sigma \sim \text{Inverse-Gamma}(2, 3) $$

$$ \text{Compute } sol_a $$

$$ data \sim \text{MvNormal}(sol_a, \sigma I) $$

**Note**: the use of truncated Gaussian for $a$ is just from my trails - I experience some bugs when $a<0$ and when $a$ is large.

In [7]:
@model bfit(x) = begin
    # Define prior
    a ~ Truncated(Normal(1.5, 1), 0.5, 2.5)  # DE param
    σ ~ InverseGamma(2, 3)                   # data noise
    
    # Update solver
    p_tmp = problem_new_parameters(prob, a); sol_tmp = solve(p_tmp,Tsit5())
    
    # Observe data
    # Here you can do a lot of ways to write the observation
    # and a lot of possible optimization to make the program faster
    # E.g. naively
    
    for i = 1:length(t)
        res = sol_tmp(t[i])
        x[:,i] ~ MvNormal(res, σ*ones(2))
    end
end

bfit (generic function with 2 methods)

## Run sampler

In [8]:
chn = sample(bfit(data), HMC(500, 0.02, 4))

[Turing]:  Assume - `a` is a parameter
  in @~(::Any, ::Any) at compiler.jl:76
[Turing]:  Assume - `σ` is a parameter
  in @~(::Any, ::Any) at compiler.jl:76
[Turing]:  Observe - `x` is an observation
  in @~(::Any, ::Any) at compiler.jl:57


[HMC] Sampling...  0%  ETA: 5:38:09
  ϵ:         0.02
  α:         1.0
[HMC] Sampling... 16%  ETA: 0:03:46[A
  ϵ:         0.02
  α:         0.9023994677893477
[HMC] Sampling... 32%  ETA: 0:01:33[A
  ϵ:         0.02
  α:         2.658201577940882e-100
[HMC] Sampling... 49%  ETA: 0:00:46[A
  ϵ:         0.02
  α:         5.866280838468989e-5
[HMC] Sampling... 65%  ETA: 0:00:24[A
  ϵ:         0.02
  α:         1.0
[HMC] Sampling... 81%  ETA: 0:00:11[A
  ϵ:         0.02
  α:         0.9985106597954047
[HMC] Sampling... 91%  ETA: 0:00:05[A
  ϵ:         0.02
  α:         5.141659514483981e-5
  pre_cond:  [1.0,1.0]

[HMC] Finished with
  Running time        = 47.387767281999935;
  Accept rate         = 0.552;
  #lf / sample        = 3.992;
  #evals / sample     = 3.996;
  pre-cond. diag mat  = [1.0,1.0].


[HMC] Sampling...100% Time: 0:00:48


Object of type "Turing.Chain"

Iterations = 1:500
Thinning interval = 1
Chains = 1
Samples per chain = 500

[-486.909 0.0 … 2.30228 0.02; -218.738 4.0 … 2.31922 0.02; … ; -55.0666 4.0 … 1.50907 0.02; -55.0666 4.0 … 1.50907 0.02]

## Make prediction 1: averaging samples as point estimate

In [9]:
a_avg = mean(chn[:a])
σ_avg = mean(chn[:σ])
println("E[a] = $a_avg, E[σ] = $σ_avg")
println("a.diff = $(a_avg - 1.5), σ.diff = $(σ_avg - 0.49)")

E[a] = 1.609196623975551, E[σ] = 0.9155962379216105
a.diff = 0.10919662397555108, σ.diff = 0.4255962379216105


In [10]:
tp = collect(linspace(0,10,200)) # t for plot
;

In [11]:
p_avg = problem_new_parameters(prob, a_avg); sol_avg = solve(p_avg,Tsit5())
pred_avg = convert(Array, VectorOfArray([sol_avg(tp[i]) for i in 1:length(tp)]))
plot_data()
plot!(tp, pred_avg[1,:], lab="#prey (prediction)")
plot!(tp, pred_avg[2,:], lab="#predator (prediction)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,294.203 57.6346,292.756 60.255,291.053 62.8754,289.093 65.4959,286.87 68.1163,284.38 70.7368,281.612 73.3572,278.555 75.9776,275.196 78.5981,271.52 
 81.2185,267.51 83.839,263.148 86.4594,258.415 89.0799,253.288 91.7003,247.746 94.3207,241.764 96.9412,235.32 99.5616,228.387 102.182,220.942 104.803,212.962 
 107.423,204.426 110.043,195.31 112.664,185.597 115.284,175.283 117.905,164.373 120.525,152.884 123.146,140.851 125.766,128.339 128.386,115.423 131.007,102.261 
 133.627,89.0903 136.248,76.2298 138.868,64.0784 141.489,53.1168 144.109,44.2135 146.73,38.3168 149.35,36.3462 151.97,39.1919 154.591,47.7132 157.211,62.3431 
 159.832,82.4402 162.452,106.75 165.073,133.486 167.693,160.452 170.314,185.858 172.934,208.431 175.554,227.556 178.175,243.296 180.795,256.055 183.416,266.223 
 186.036,274.247 188.657,280.522 191.277,285.4 193.898,289.156 196.518,292.01 199.138,294.132 201.759,295.648 204.379,296.646 207,297.209 209.62,297.391 
 212.241,297.234 214.861,296.772 217.481,296.024 220.102,295.006 222.722,293.725 225.343,292.186 227.963,290.39 230.584,288.337 233.204,286.02 235.825,283.432 
 238.445,280.563 241.065,277.4 243.686,273.93 246.306,270.137 248.927,266.005 251.547,261.513 254.168,256.642 256.788,251.37 259.409,245.674 262.029,239.531 
 264.649,232.916 267.27,225.804 269.89,218.173 272.511,209.999 275.131,201.263 277.752,191.937 280.372,182.009 282.993,171.477 285.613,160.354 288.233,148.664 
 290.854,136.447 293.474,123.798 296.095,110.804 298.715,97.5995 301.336,84.4461 303.956,71.7272 306.576,59.9501 309.197,49.7457 311.817,41.8682 314.438,37.1955 
 317.058,36.7263 319.679,41.5439 322.299,52.2647 324.92,68.876 327.54,90.7374 330.16,116.242 332.781,143.236 335.401,169.78 338.022,194.192 340.642,215.501 
 343.263,233.465 345.883,248.136 348.504,259.842 351.124,269.161 353.744,276.523 356.365,282.278 358.985,286.746 361.606,290.178 364.226,292.759 366.847,294.651 
 369.467,295.97 372.088,296.806 374.708,297.231 377.328,297.293 379.949,297.028 382.569,296.463 385.19,295.617 387.81,294.502 390.431,293.129 393.051,291.499 
 395.671,289.614 398.292,287.467 400.912,285.054 403.533,282.365 406.153,279.39 408.774,276.117 411.394,272.532 414.015,268.618 416.635,264.356 419.255,259.729 
 421.876,254.715 424.496,249.293 427.117,243.439 429.737,237.128 432.358,230.338 434.978,223.043 437.599,215.218 440.219,206.838 442.839,197.885 445.46,188.345 
 448.08,178.21 450.701,167.479 453.321,156.223 455.942,144.415 458.562,132.048 461.183,119.199 463.803,106.029 466.423,92.7849 469.044,79.7972 471.664,67.4812 
 474.285,56.3366 476.905,46.9476 479.526,40.0682 482.146,36.8551 484.767,38.3444 487.387,45.2845 490.007,58.1384 492.628,76.8599 495.248,100.134 497.869,126.221 
 500.489,153.206 503.11,179.054 505.73,202.377 508.35,222.496 510.971,239.166 513.591,252.59 516.212,263.378 518.832,271.949 521.453,278.683 524.073,283.939 
 526.694,288 529.314,291.093 531.934,293.406 534.555,295.076 537.175,296.211 539.796,296.894 542.416,297.182 545.037,297.12 547.657,296.741 550.278,296.069 
 552.898,295.121 555.518,293.909 558.139,292.44 560.759,290.714 563.38,288.73 566,286.481 568.621,283.963 571.241,281.165 573.862,278.077 576.482,274.686 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#c271d2; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,294.203 57.6346,298.289 60.255,301.923 62.8754,305.15 65.4959,308.01 68.1163,310.543 70.7368

## Make prediction 2: the Bayesian way

In [12]:
all_a = chn[:a]
p1 = []
p2 = []
for i = 1:10:length(all_a)    # not taking each sample as that's too expensive
    p_bayes = problem_new_parameters(prob, all_a[i]); sol_bayes = solve(p_bayes,Tsit5())
    pred_bayes = convert(Array, VectorOfArray([sol_avg(tp[i]) for i in 1:length(tp)]))
    push!(p1, pred_bayes[1,:])
    push!(p2, pred_bayes[2,:])
end
p1 = convert(Array,VectorOfArray(p1))
p2 = convert(Array,VectorOfArray(p2))
p1 = Array{Float64,2}(p1)
p2 = Array{Float64,2}(p2)
;

### Compute mean and 3 std error bar

In [13]:
m1 = vec(mean(p1, 2))
m2 = vec(mean(p2, 2))
e1 = 3*vec(sqrt(var(p1,2)))
e2 = 3*vec(sqrt(var(p2,2)))
;

In [14]:
plot_data()
plot!(tp, m1, ribbon=e1, lab="#prey (prediction)")
plot!(tp, m2, ribbon=e2, lab="#predator (prediction)")
# plot!(tp, m1, ribbon=e1*1e14, lab="#prey (prediction)")
# plot!(tp, m2, ribbon=e2*1e14, lab="#predator (prediction)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip02)" points="
55.0141,294.203 57.6346,292.756 60.255,291.053 62.8754,289.093 65.4959,286.87 68.1163,284.38 70.7368,281.612 73.3572,278.555 75.9776,275.196 78.5981,271.52 
 81.2185,267.51 83.839,263.148 86.4594,258.415 89.0799,253.288 91.7003,247.746 94.3207,241.764 96.9412,235.32 99.5616,228.387 102.182,220.942 104.803,212.962 
 107.423,204.426 110.043,195.31 112.664,185.597 115.284,175.283 117.905,164.373 120.525,152.884 123.146,140.851 125.766,128.339 128.386,115.423 131.007,102.261 
 133.627,89.0903 136.248,76.2298 138.868,64.0784 141.489,53.1168 144.109,44.2135 146.73,38.3168 149.35,36.3462 151.97,39.1919 154.591,47.7132 157.211,62.3431 
 159.832,82.4402 162.452,106.75 165.073,133.486 167.693,160.452 170.314,185.858 172.934,208.431 175.554,227.556 178.175,243.296 180.795,256.055 183.416,266.223 
 186.036,274.247 188.657,280.522 191.277,285.4 193.898,289.156 196.518,292.01 199.138,294.132 201.759,295.648 204.379,296.646 207,297.209 209.62,297.391 
 212.241,297.234 214.861,296.772 217.481,296.024 220.102,295.006 222.722,293.725 225.343,292.186 227.963,290.39 230.584,288.337 233.204,286.02 235.825,283.432 
 238.445,280.563 241.065,277.4 243.686,273.93 246.306,270.137 248.927,266.005 251.547,261.513 254.168,256.642 256.788,251.37 259.409,245.674 262.029,239.531 
 264.649,232.916 267.27,225.804 269.89,218.173 272.511,209.999 275.131,201.263 277.752,191.937 280.372,182.009 282.993,171.477 285.613,160.354 288.233,148.664 
 290.854,136.447 293.474,123.798 296.095,110.804 298.715,97.5995 301.336,84.4461 303.956,71.7272 306.576,59.9501 309.197,49.7457 311.817,41.8682 314.438,37.1955 
 317.058,36.7263 319.679,41.5439 322.299,52.2647 324.92,68.876 327.54,90.7374 330.16,116.242 332.781,143.236 335.401,169.78 338.022,194.192 340.642,215.501 
 343.263,233.465 345.883,248.136 348.504,259.842 351.124,269.161 353.744,276.523 356.365,282.278 358.985,286.746 361.606,290.178 364.226,292.759 366.847,294.651 
 369.467,295.97 372.088,296.806 374.708,297.231 377.328,297.293 379.949,297.028 382.569,296.463 385.19,295.617 387.81,294.502 390.431,293.129 393.051,291.499 
 395.671,289.614 398.292,287.467 400.912,285.054 403.533,282.365 406.153,279.39 408.774,276.117 411.394,272.532 414.015,268.618 416.635,264.356 419.255,259.729 
 421.876,254.715 424.496,249.293 427.117,243.439 429.737,237.128 432.358,230.338 434.978,223.043 437.599,215.218 440.219,206.838 442.839,197.885 445.46,188.345 
 448.08,178.21 450.701,167.479 453.321,156.223 455.942,144.415 458.562,132.048 461.183,119.199 463.803,106.029 466.423,92.7849 469.044,79.7972 471.664,67.4812 
 474.285,56.3366 476.905,46.9476 479.526,40.0682 482.146,36.8551 484.767,38.3444 487.387,45.2845 490.007,58.1384 492.628,76.8599 495.248,100.134 497.869,126.221 
 500.489,153.206 503.11,179.054 505.73,202.377 508.35,222.496 510.971,239.166 513.591,252.59 516.212,263.378 518.832,271.949 521.453,278.683 524.073,283.939 
 526.694,288 529.314,291.093 531.934,293.406 534.555,295.076 537.175,296.211 539.796,296.894 542.416,297.182 545.037,297.12 547.657,296.741 550.278,296.069 
 552.898,295.121 555.518,293.909 558.139,292.44 560.759,290.714 563.38,288.73 566,286.481 568.621,283.963 571.241,281.165 573.862,278.077 576.482,274.686 
 576.482,274.686 573.862,278.077 571.241,281.165 568.621,283.963 566,286.481 563.38,288.73 560.759,290.714 558.139,292.44 555.518,293.909 552.898,295.121 
 550.278,296.069 547.657,296.741 545.037,297.12 542.416,297.182 539.796,296.894 537.175,296.211 534.555,295.076 531.934,293.406 529.314

**Note**: the prediction variance seems to be very small; can't even see from the plot

## Also check the trace and density plot of a and $\sigma$

In [15]:
samples_a = chn[:a]
samples_σ = chn[:σ]
;

In [16]:
plot([samples_a, samples_a], layout=@layout([a; b]),
      seriestype=[:line :hist], nbins=100,
      labels=["a (samples)", "a (ground true)"])
plot!([1.5, 1.5], layout=@layout([a; b]),  
       seriestype = [:hline, :vline],
       labels=["a (samples)", "a (ground true)"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 1.6 
 
 
 1.8 
 
 
 2.0 
 
 
 2.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,14.4588 40.4778,11.2685 41.5855,12.0655 42.6933,10.7174 43.801,10.7127 44.9087,11.4809 46.0164,10.3631 47.1242,8.92021 48.2319,11.6753 49.3396,17.7395 
 50.4474,15.7808 51.5551,16.596 52.6628,16.7125 53.7705,14.2642 54.8783,17.0566 55.986,15.1131 57.0937,7.87402 58.2014,8.57654 59.3092,10.0805 60.4169,12.7422 
 61.5246,13.8264 62.6324,16.9501 63.7401,15.4 64.8478,16.5548 65.9555,17.6032 67.0633,18.2855 68.171,21.3466 69.2787,22.7906 70.3864,23.7532 71.4942,28.4613 
 72.6019,33.6215 73.7096,33.6215 74.8174,36.593 75.9251,31.2956 77.0328,35.1653 78.1405,35.1478 79.2483,37.7595 80.356,37.7595 81.4637,37.7595 82.5714,36.327 
 83.6792,37.1392 84.7869,36.8127 85.8946,36.3559 87.0024,37.4926 88.1101,31.2854 89.2178,32.4002 90.3255,28.1391 91.4333,24.6843 92.541,26.6873 93.6487,30.1069 
 94.7564,23.413 95.8642,31.2251 96.9719,36.5287 98.0796,36.5287 99.1874,34.0528 100.295,31.0397 101.403,27.6538 102.511,28.4136 103.618,29.1462 104.726,25.7141 
 105.834,28.0753 106.941,34.7338 108.049,31.4907 109.157,38.9994 110.265,41.1357 111.372,41.1357 112.48,48.6544 113.588,51.0073 114.696,61.6294 115.803,68.4096 
 116.911,76.496 118.019,84.5036 119.126,87.8011 120.234,89.1678 121.342,97.5789 122.45,109.273 123.557,125.998 124.665,137.295 125.773,148.019 126.881,161.147 
 127.988,168.504 129.096,162.429 130.204,163.497 131.311,163.497 132.419,164.659 133.527,165.223 134.635,165.421 135.742,165.421 136.85,166.856 137.958,165.443 
 139.066,166.625 140.173,166.625 141.281,166.625 142.389,166.625 143.496,166.625 144.604,166.625 145.712,166.625 146.82,165.25 147.927,165.25 149.035,165.547 
 150.143,165.547 151.251,165.547 152.358,165.547 153.466,165.436 154.574,165.436 155.681,165.99 156.789,165.99 157.897,164.437 159.005,164.437 160.112,164.437 
 161.22,164.906 162.328,164.906 163.436,164.906 164.543,164.906 165.651,164.408 166.759,166.918 167.866,166.918 168.974,166.918 170.082,166.918 171.19,166.918 
 172.297,164.267 173.405,165.082 174.513,165.082 175.621,164.998 176.728,164.95 177.836,164.95 178.944,165.467 180.051,165.168 181.159,165.168 182.267,165.168 
 183.375,165.168 184.482,164.164 185.59,163.723 186.698,165.486 187.806,165.615 188.913,165.481 190.021,163.774 191.129,163.815 192.236,166.123 193.344,166.123 
 194.452,166.123 195.56,162.205 196.667,165.656 197.775,165.285 198.883,165.285 199.991,165.455 201.098,166.838 202.206,165.094 203.314,165.094 204.421,165.094 
 205.529,165.411 206.637,165.411 207.745,165.411 208.852,165.326 209.96,165.326 211.068,165.326 212.176,165.326 213.283,165.326 214.391,165.326 215.499,165.764 
 216.606,165.764 217.714,165.764 218.822,165.764 219.93,165.764 221.037,165.764 222.145,165.764 223.253,165.764 224.361,165.764 225.468,165.764 226.576,166.192 
 227.684,165.056 228.791,165.62 229.899,165.212 231.007,166.223 232.115,166.223 233.222,166.223 234.33,166.223 235.438,166.223 236.546,166.223 237.653,166.223 
 238.761,166.223 239.869,165.504 240.976,165.504 242.084,166.636 243.192,166.229 244.3,164.682 245.407,165.467 246.515,165.487 247.623,164.395 248.731,166.524 
 249.838,166.524 250.946,167.094 252.054,167.802 253.161,165.457 254.269,165.457 255.377,164.718 256.485,164.718 257.592,164.591 258.7,164.591 259.808,164.591 
 260.916,166.096 262.023,166.096 263.131,166.15 264.239,165.282 265.346,165.817 266.454,164.344 267.562,164.344 268.67,164.344 269.777,164.344 270.885,164.344 
 271.993,168.015 273.101,168.015 274.208,167.353 275.316,166.881 276.424,163.78 277.531,163.78 278.639,167.593 279.747,167.593 280.855,165.776 281.962,165.776 
 283.07,165.776 284.178,165.776 285.286,165.776 286.393,164.844 287.501,164.844 288.609,164.844 289.716,164.844 290.824,164.844 291.932,164.844 293.04,164.844

In [17]:
plot([samples_σ, samples_σ], layout=@layout([a; b]),
      seriestype=[:line :hist], nbins=100,
      labels=["sigma (samples)", "sigma (ground true)"])
plot!([0.49, 0.49], layout=@layout([a; b]),  
       seriestype = [:hline, :vline],
       labels=["sigma (samples)", "sigma (ground true)"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,143.695 40.4778,7.87402 41.5855,39.3926 42.6933,56.7333 43.801,46.4343 44.9087,50.98 46.0164,55.1134 47.1242,54.2635 48.2319,46.3069 49.3396,43.5644 
 50.4474,46.9703 51.5551,50.1017 52.6628,42.3329 53.7705,50.9325 54.8783,41.7199 55.986,63.0314 57.0937,65.7307 58.2014,60.6809 59.3092,52.9102 60.4169,62.6045 
 61.5246,67.8501 62.6324,78.8366 63.7401,57.9569 64.8478,42.531 65.9555,39.1398 67.0633,56.9146 68.171,65.2466 69.2787,63.664 70.3864,59.9412 71.4942,54.6091 
 72.6019,55.7366 73.7096,55.7366 74.8174,72.503 75.9251,57.4653 77.0328,54.2113 78.1405,45.4829 79.2483,48.5919 80.356,48.5919 81.4637,48.5919 82.5714,44.1073 
 83.6792,56.4115 84.7869,47.1867 85.8946,40.5078 87.0024,49.5477 88.1101,45.5661 89.2178,65.7466 90.3255,48.8781 91.4333,47.0754 92.541,55.6752 93.6487,45.0525 
 94.7564,47.0381 95.8642,61.761 96.9719,51.3835 98.0796,51.3835 99.1874,67.8829 100.295,66.9595 101.403,57.6904 102.511,45.2546 103.618,47.333 104.726,48.7733 
 105.834,52.6988 106.941,67.7768 108.049,53.5113 109.157,42.8895 110.265,40.3517 111.372,40.3517 112.48,44.226 113.588,64.6362 114.696,63.6281 115.803,59.932 
 116.911,68.4093 118.019,76.7812 119.126,66.6389 120.234,75.6644 121.342,71.3525 122.45,74.754 123.557,86.922 124.665,96.721 125.773,105.707 126.881,122.743 
 127.988,139.086 129.096,149.373 130.204,156.557 131.311,156.557 132.419,155.566 133.527,158.796 134.635,162.363 135.742,162.363 136.85,163.848 137.958,165.053 
 139.066,167.349 140.173,167.349 141.281,167.349 142.389,167.349 143.496,167.349 144.604,167.349 145.712,167.349 146.82,165.606 147.927,165.606 149.035,164.97 
 150.143,164.97 151.251,164.97 152.358,164.97 153.466,164.288 154.574,164.288 155.681,166.276 156.789,166.276 157.897,166.692 159.005,166.692 160.112,166.692 
 161.22,166.944 162.328,166.944 163.436,166.944 164.543,166.944 165.651,164.96 166.759,165.699 167.866,165.699 168.974,165.699 170.082,165.699 171.19,165.699 
 172.297,165.366 173.405,164.223 174.513,164.223 175.621,161.624 176.728,160.644 177.836,160.644 178.944,164.641 180.051,162.667 181.159,162.667 182.267,162.667 
 183.375,162.667 184.482,162.03 185.59,160.988 186.698,162.599 187.806,164.856 188.913,165.994 190.021,163.412 191.129,163.071 192.236,162.663 193.344,162.663 
 194.452,162.663 195.56,162.365 196.667,162.753 197.775,165.342 198.883,165.342 199.991,164.705 201.098,165.299 202.206,166.595 203.314,166.595 204.421,166.595 
 205.529,167.991 206.637,167.991 207.745,167.991 208.852,168.345 209.96,168.345 211.068,168.345 212.176,168.345 213.283,168.345 214.391,168.345 215.499,167.832 
 216.606,167.832 217.714,167.832 218.822,167.832 219.93,167.832 221.037,167.832 222.145,167.832 223.253,167.832 224.361,167.832 225.468,167.832 226.576,164.587 
 227.684,166.237 228.791,164.215 229.899,162.834 231.007,166.777 232.115,166.777 233.222,166.777 234.33,166.777 235.438,166.777 236.546,166.777 237.653,166.777 
 238.761,166.777 239.869,166.594 240.976,166.594 242.084,165.337 243.192,164.721 244.3,165.104 245.407,166.065 246.515,166.073 247.623,166.235 248.731,164.706 
 249.838,164.706 250.946,163.248 252.054,160.509 253.161,161.689 254.269,161.689 255.377,163.768 256.485,163.768 257.592,163.202 258.7,163.202 259.808,163.202 
 260.916,164.905 262.023,164.905 263.131,165.023 264.239,165.513 265.346,165.236 266.454,165.987 267.562,165.987 268.67,165.987 269.777,165.987 270.885,165.987 
 271.993,164.042 273.101,164.042 274.208,163.261 275.316,163.904 276.424,165.922 277.531,165.922 278.639,165.292 279.747,165.292 280.855,167.324 281.962,167.324 
 283.07,167.324 284.178,167.324 285.286,167.324 286.393,166.959 287.501,166.959 288.609,166.959 289.716,166.959 290.824,166.959 291.932,166.959 293.04,166.959 
 294.1